In [ ]:
import pandas as pd
import numpy as np

# ------------------------------------------------
# 1. Load and use the correct header row
# ------------------------------------------------
df = pd.read_csv("WholeYear.csv", header=1)

# Strip whitespace from all column names
df.columns = df.columns.str.strip()

# ------------------------------------------------
# 2. Convert Time column to datetime
# ------------------------------------------------
df["Time (LST)"] = pd.to_datetime(df["Time (LST)"], errors="coerce")

# ------------------------------------------------
# 3. Drop rows where ALL measurement fields are NaN
# ------------------------------------------------
df = df.dropna(how="all")

# ------------------------------------------------
# 4. Convert all numeric columns to numeric (except time)
# ------------------------------------------------
for col in df.columns:
    if col != "Time (LST)":
        df[col] = pd.to_numeric(df[col], errors="coerce")

# ------------------------------------------------
# 5. Drop columns that are entirely NaN
# ------------------------------------------------
df = df.dropna(axis=1, how="all")
# 2. Drop the two specified columns
cols_to_drop = ["Heat Index (°F)", "Wind Chill (°F)","Barometric Pressure (in Hg)"]
df = df.drop(columns=cols_to_drop, errors="ignore")

# ------------------------------------------------
# 6. Save cleaned dataset
# ------------------------------------------------
df.to_csv("WholeYear_Cleaned.csv", index=False)

print("Cleaning complete.")
print("New shape:", df.shape)
print("Saved as WholeYear_Cleaned.csv")


Cleaning complete.
New shape: (5088, 24)
Saved as Table_Hrly_ACRE_clean.csv


/var/folders/1b/9b4m32_9491bc76v2y8h5vt80000gn/T/ipykernel_29734/3136797146.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Time (LST)"] = pd.to_datetime(df["Time (LST)"], errors="coerce")


In [2]:
import pandas as pd
import numpy as np

# -------------------------------
# 1. Load CLEANED dataset
# -------------------------------
df['Time (LST)'] = pd.to_datetime(df['Time (LST)'])
df = df.set_index('Time (LST)')


# -------------------------------
# 2. REMOVE columns you dropped earlier
# -------------------------------
cols_to_drop = ["Heat Index (°F)", "Wind Chill (°F)", "Barometric Pressure (in Hg)"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


# -------------------------------
# 3. Interpolate missing values
# -------------------------------
df_interp = df.interpolate(method="time")
df_interp = df_interp.bfill().ffill()


# -------------------------------
# 4. Difference only non-stationary variables
# (from your previous ADF results)
# -------------------------------
non_stationary_vars = [
    '2" Soil Water Content (%)',
    '8" Soil Water Content (%)',
    '20" Soil Water Content (%)',
    '4" Soil Water Content (%)'
]

df_stationary = df_interp.copy()

for col in non_stationary_vars:
    df_stationary[col] = df_stationary[col].diff()

# Drop first row (became NaN from differencing)
df_stationary = df_stationary.dropna()


# -------------------------------
# 5. Define NEW TARGET for this study
# -------------------------------
Y2_var = '20" Soil Temp (°F)'     # NEW Y
X2_vars = [col for col in df_stationary.columns if col != Y2_var]

print("Initialization complete.")
print("Target variable Y2:", Y2_var)
print("Number of X variables:", len(X2_vars))
print("Columns ready:", df_stationary.columns.tolist())


Initialization complete.
Target variable Y2: 20" Soil Temp (°F)
Number of X variables: 22
Columns ready: ['Air Temp (°F)', '0.5 m Air Temp (°F)', '1.5 m Air Temp (°F)', '3 m Air Temp (°F)', 'Relative Humidity (%)', 'Precipitation (in)', 'Accumulated Precip (in)', 'Solar Radiation (W / m²)', 'Wind Speed (mph)', 'Wind Direction (°)', 'Wind Gust (mph)', '4" Bare Soil Temp (°F)', '4" Grass Soil Temp (°F)', '2" Soil Temp (°F)', '2" Soil Water Content (%)', '4" Soil Temp (°F)', '4" Soil Water Content (%)', '8" Soil Temp (°F)', '8" Soil Water Content (%)', '20" Soil Temp (°F)', '20" Soil Water Content (%)', 'Inversion Strength', 'Max Inversion']


In [3]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
import warnings
warnings.filterwarnings("ignore")

best_lags_20T = {}
p_values_20T = {}
importance_20T = {}

Y = df_stationary[Y2_var]

print("\n==== Granger Lag Selection and Importance for 20\" Soil Temp (°F) ====\n")

# Function to build lagged dataset for OLS
def build_lagged_df(Y, X, lag):
    data = pd.DataFrame({"Y": Y, "X": X})
    for i in range(1, lag+1):
        data[f"Y_lag{i}"] = Y.shift(i)
        data[f"X_lag{i}"] = X.shift(i)
    return data.dropna()

# ---------------------------------------------------
# 1. AIC-based lag selection for each X variable
# ---------------------------------------------------
for X_var in X2_vars:
    print(f"\n--- Processing {X_var} ---")

    X = df_stationary[X_var]
    aic_scores = {}

    for lag in range(1, 13):  # test lags 1–12
        try:
            d = build_lagged_df(Y, X, lag)
            Y_dep = d["Y"]
            X_regs = sm.add_constant(d.drop(columns=["Y"]))
            model = sm.OLS(Y_dep, X_regs).fit()
            aic_scores[lag] = model.aic
        except:
            pass

    if len(aic_scores) == 0:
        print("  No valid lags. Skipping.")
        continue

    best_lag = min(aic_scores, key=aic_scores.get)
    best_lags_20T[X_var] = best_lag

    print(f"  Best lag (AIC): {best_lag}")

    # ---------------------------------------------------
    # 2. Granger Test using identified best lag
    # ---------------------------------------------------
    try:
        data_pair = df_stationary[[Y2_var, X_var]]
        result = grangercausalitytests(data_pair, maxlag=best_lag, verbose=False)

        # extract F-test
        f_test = result[best_lag][0]['ssr_ftest']
        F_stat, p_val = f_test[0], f_test[1]

        p_values_20T[X_var] = p_val
        importance_20T[X_var] = (
            "Important for forecasting Y₂" if p_val < 0.05 else "Not important for forecasting Y₂"
        )

        print(f"  F-statistic: {F_stat:.4f}")
        print(f"  p-value: {p_val:.4f}")
        print(f"  Interpretation: {importance_20T[X_var]}")

    except Exception as e:
        print(f"  Granger test failed: {e}")

# ---------------------------------------------------
# 3. Summary
# ---------------------------------------------------
print("\n==== Summary ====\n")
print("best_lags_20T =", best_lags_20T)
print("\np_values_20T =", p_values_20T)
print("\nimportance_20T =", importance_20T)



==== Granger Lag Selection and Importance for 20" Soil Temp (°F) ====


--- Processing Air Temp (°F) ---
  Best lag (AIC): 11
  F-statistic: 24.1397
  p-value: 0.0000
  Interpretation: Important for forecasting Y₂

--- Processing 0.5 m Air Temp (°F) ---
  Best lag (AIC): 11
  F-statistic: 23.3334
  p-value: 0.0000
  Interpretation: Important for forecasting Y₂

--- Processing 1.5 m Air Temp (°F) ---
  Best lag (AIC): 11
  F-statistic: 22.5570
  p-value: 0.0000
  Interpretation: Important for forecasting Y₂

--- Processing 3 m Air Temp (°F) ---
  Best lag (AIC): 11
  F-statistic: 21.9349
  p-value: 0.0000
  Interpretation: Important for forecasting Y₂

--- Processing Relative Humidity (%) ---
  Best lag (AIC): 12
  F-statistic: 22.0198
  p-value: 0.0000
  Interpretation: Important for forecasting Y₂

--- Processing Precipitation (in) ---
  Best lag (AIC): 7
  F-statistic: 1.1717
  p-value: 0.3154
  Interpretation: Not important for forecasting Y₂

--- Processing Accumulated Precip (in)